In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import biom
import os
import scipy.stats
from statsmodels.sandbox.stats.multicomp import fdrcorrection0

In [2]:
sample_md_fp = 'combined-map.tsv'
sample_md = pd.read_csv(sample_md_fp, sep='\t', index_col=0, dtype=object)


In [3]:
data_dir = "../microbiome-data/run1-4/vsearch-100/cd_even5721/"
tax_tables = []
for level in range(2,7):
    l_fp = os.path.join(data_dir, "taxa_plots/table_mc5721_sorted_L%d.txt" % level)
    l_df = pd.read_csv(l_fp, sep='\t', skiprows=1, index_col=0)
    l_df.index = l_df.index.astype(str)
    tax_tables.append(l_df.T)
tax_table = pd.concat(tax_tables, axis=1)
sample_md = sample_md.merge(tax_table, left_index=True, right_index=True)


In [4]:
from asd import filter_sample_md
initial_asd_stool = filter_sample_md(sample_md, includes=[('Group', 'autism'), ('SampleType', 'stool'), 
                                                          ('time_point', '1'), ('responder', 'Responder')])
initial_asd_stool = initial_asd_stool.set_index('SubjectID')

final_asd_stool = filter_sample_md(sample_md, includes=[('Group', 'autism'), ('SampleType', 'stool'), 
                                                        ('time_point', '4'), ('responder', 'Responder')])
final_asd_stool = final_asd_stool.set_index('SubjectID')

In [5]:
donor_stool = filter_sample_md(sample_md, includes=[('SampleType', 'donor-stool')])

In [6]:
_data = []
for tax in tax_table.columns:
    initial_asd_median = initial_asd_stool[tax].median()
    final_asd_median = final_asd_stool[tax].median()
    donor_median = donor_stool[tax].median()
    if (initial_asd_median < donor_median) and (final_asd_median > initial_asd_median):
        t, p = scipy.stats.ttest_1samp(final_asd_stool[tax] - initial_asd_stool[tax], popmean=0)
        if t > 0:
            p = p/2
        else:
            p = 1.0
        try:
            fold_donor_enrichment = donor_median/initial_asd_median
            fold_final_enrichment = final_asd_median/initial_asd_median
        except ZeroDivisionError:
            fold_donor_enrichment = fold_final_enrichment = np.inf
        _data.append([tax, fold_donor_enrichment, fold_final_enrichment, donor_median, 
                      initial_asd_median, final_asd_median, t, p])
tax_engraftment_df = pd.DataFrame(_data, columns=['Taxonomy', 'Fold donor enrichment', 
                                                  'Fold final enrichment',
                                                  'Donor median abundance',
                                                  'Initial ASD median abundance', 
                                                  'Final ASD median abundance', 't (one-tailed, one-sample)', 
                                                  'p-value']).set_index('Taxonomy')
tax_engraftment_df['FDR p-value'] = fdrcorrection0(tax_engraftment_df['p-value'])[1]
tax_engraftment_df.sort('Fold final enrichment', ascending=False).to_csv('taxonomy-engraftment.csv')

/Users/caporaso/miniconda3/envs/q2d2/lib/python3.5/site-packages/ipykernel/__main__.py:26: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


The results of this analysis are in a [Google Sheet](https://docs.google.com/spreadsheets/d/1iwU6wQ9JApx7I6H4D53XFDFRu_Yige2HaiC88YkjqW0/edit?usp=sharing).

In [7]:
alpha = 0.01
_data = []
for tax in tax_table.columns:
    initial_asd_median = initial_asd_stool[tax].median()
    final_asd_median = final_asd_stool[tax].median()
    donor_median = donor_stool[tax].median()
    if final_asd_median > 0 or initial_asd_median > 0:
        t, p = scipy.stats.ttest_1samp(final_asd_stool[tax] - initial_asd_stool[tax], popmean=0)
        try:
            fold_donor_enrichment = donor_median/initial_asd_median
            fold_final_enrichment = final_asd_median/initial_asd_median
        except ZeroDivisionError:
            fold_donor_enrichment = fold_final_enrichment = np.inf
        _data.append([tax, fold_donor_enrichment, fold_final_enrichment, donor_median, 
                      initial_asd_median, final_asd_median, t, p])
tax_change_df = pd.DataFrame(_data, columns=['Taxonomy', 'Fold donor enrichment', 
                                                  'Fold final enrichment',
                                                  'Donor median abundance',
                                                  'Initial ASD median abundance', 
                                                  'Final ASD median abundance', 't (two-tailed, one-sample)', 
                                                  'p-value']).set_index('Taxonomy')
tax_change_df['FDR p-value'] = fdrcorrection0(tax_change_df['p-value'])[1]
tax_change_df.sort('FDR p-value', ascending=True).to_csv('taxonmy-change.csv')

/Users/caporaso/miniconda3/envs/q2d2/lib/python3.5/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
